# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset from data folder or link provided in challenge 1
pokemon = pd.read_csv('data/pokemon_updated.csv',sep=',')
# pokemon.head(5)
# pokemon.groupby(['Name'])['HP','Attack'].sum()
pokemon['Total 2'] = pokemon[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].sum(axis=1)
pokemon.head(5)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total 2
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [3]:
# your code here

# Total is constructed by the sum of all different abilities
pokemon['Total 2'] = pokemon[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].sum(axis=1)

# If my hypothesis is incorrect the below query will generate a number above > 0
print('Instances whereby the sum of abilities does not equal Total:',len(pokemon[pokemon['Total 2'] != pokemon['Total']]))


#print remove extra comparison column
pokemon = pokemon.drop(columns='Total 2')

Instances whereby the sum of abilities does not equal Total: 0


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

[One Hot Encoding](https://drive.google.com/file/d/1McE3eNAyUxBXHfEvyTTz-ZrMDB7-U3_z/view?usp=sharing)

In [4]:
# your code here
pokemon2 = pd.concat([pd.get_dummies(pokemon[col]) for col in pokemon[['Type 1','Type 2']]],axis=1)
pokemon2.head().T

,0,1,2,3,4
Bug,0,0,0,0,0
Dark,0,0,0,0,0
Dragon,0,0,0,0,0
Electric,0,0,0,0,0
Fairy,0,0,0,0,0
Fighting,0,0,0,0,0
Fire,0,0,0,0,1
Flying,0,0,0,0,0
Ghost,0,0,0,0,0
Grass,1,1,1,1,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [5]:
pokemon2.corrwith(pokemon['Total']).sort_values(ascending=False).index[0]

'Dragon'

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [6]:
# your code here
import re
##first I normalize the names
pokemon.Name = pd.DataFrame(pokemon.Name.apply(lambda x: re.sub(r'(.+)(Mega.+)',r'\2', x)))

#create combo types
pokemon['Combo Type'] = np.where(pokemon['Type 2'].isna(), pokemon['Type 1'], pokemon['Type 1'] + '-' + pokemon['Type 2'])

#create dummy values 
pokemon_dummies = pd.get_dummies(pokemon['Combo Type'])

#prints conclusion
print('Based on correlaction I would pick a {} pokemon'.format(pokemon_dummies.corrwith(pokemon['Total']).sort_values(ascending=False).index[0]))

# pd.concat([pd.get_dummies(pokemon[col]) for col in pokemon[['Type 1','Type 2']]],axis=1)

Based on correlaction I would pick a Dragon-Flying pokemon


In [7]:
pokemon.nlargest(5, ['Total'])

### based on the top 5 pokemons with highest scores I would pick Mewto

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Combo Type
163,Mega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True,Psychic-Fighting
164,Mega Mewtwo Y,Psychic,NaN,780,106,150,70,194,120,140,1,True,Psychic
426,Mega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,Dragon-Flying
422,KyogrePrimal Kyogre,Water,NaN,770,100,150,90,180,160,90,3,True,Water
424,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True,Ground-Fire


In [8]:
pokemon.groupby(['Combo Type']).mean().sort_values('Total',ascending=False).nlargest(10, ['Total'])
## However it seems that based on averages the ground fire pokemon score highest 
##however it seems that all dragon combo types score well. 



,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
Combo Type,,,,,,,,,
Ground-Fire,770.000000,100.000000,180.000000,160.000000,150.000000,90.000000,90.000000,3.000000,1.000000
Dragon-Ice,686.666667,125.000000,140.000000,93.333333,140.000000,93.333333,95.000000,5.000000,1.000000
Steel-Dragon,680.000000,100.000000,120.000000,120.000000,150.000000,100.000000,90.000000,4.000000,1.000000
Ghost-Dragon,680.000000,150.000000,110.000000,110.000000,110.000000,110.000000,90.000000,4.000000,1.000000
Dragon-Fire,680.000000,100.000000,120.000000,100.000000,150.000000,120.000000,90.000000,5.000000,1.000000
Dragon-Electric,680.000000,100.000000,150.000000,120.000000,120.000000,100.000000,90.000000,5.000000,1.000000
Psychic-Dark,680.000000,80.000000,160.000000,60.000000,170.000000,130.000000,80.000000,6.000000,1.000000
Dragon-Psychic,650.000000,80.000000,100.000000,97.500000,135.000000,127.500000,110.000000,3.000000,1.000000
Rock-Dark,650.000000,100.000000,149.000000,130.000000,95.000000,110.000000,66.000000,2.000000,0.000000


In [9]:
# The Ground Fire combo type only includes one pokemon 
print('Number of Ground Fire pokemons: {}'.format(len(pokemon[pokemon['Combo Type'] == 'Ground-Fire'])))



Number of Ground Fire pokemons: 1
